# Comparing simulations

In this notebook you will compare different simulations, to understand how a manipulation impacts the network activity.

In particular, we compare 4 different simulations:
1. all minis activated ("minis")
2. disable minis from internal synapses and leave only minis from Schaffer collaterals synapses ("proj_minis")
3. disable minis from Schaffer collaterals synapses and leave minis from internal synapses ("internal_minis")
4. disable all the minis ("no_minis")

We expect that firing rates decrease on average from simulation 1 to 4.
If this is not the case, it means there is some problems in the network or in the simulation we have to fix.

---

Import some python packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# package to work with the circuit
from bluepysnap import Simulation
from bluepysnap.bbp import Cell

Specify where we have the simulations (we create the simulation objects later) and set parameters for the analysis

In [ ]:
root = '/home/data-bbp/20191017/simulations/activate_minis/'

folders = ('minis', 'proj_minis', 'internal_minis', 'no_minis')

target = 'CA1' # group of cells to be analised
sample = 4000 # number of cells to be analised
t_start, t_end = 200, 1000 # exclude initial transient

Define an helper function to compute the mean frequency of a give gid

In [ ]:
def get_spike_frequency(sim, gid, t_start, t_end):
    table = sim.spikes.filter([gid], t_start=t_start, t_stop=t_end).report
    if (t_end-t_start) == 0:
        return 0
    return len(table)*1000.0/(t_end-t_start)

List the morphological types (m-types) that will be analised

In [ ]:
sim = Simulation(root + folders[0] + '/simulation_config.json')
mtypes = sim.circuit.nodes["hippocampus_neurons"].property_values(Cell.MTYPE)
mtypes = sorted(mtypes)

Initialise table to host the results

In [ ]:
df = pd.DataFrame(index=mtypes,columns=folders)
df

Analise the four simulations and store the results in the table

In [ ]:
for folder in folders:
    path = root + folder + '/simulation_config.json'
    sim = Simulation(path)
    for mtype in mtypes:
        print(folder,mtype)
        gids = sim.circuit.nodes["hippocampus_neurons"].ids(group={Cell.MTYPE: mtype, "$node_set": target},limit=sample)
        freqs = np.array([get_spike_frequency(sim, gid, t_start, t_end) for gid in gids])
        df.loc[mtype,folder] = freqs.mean()
df

Plot the results

In [ ]:
fig, ax = plt.subplots(figsize=(11.69, 8.27))
data = df.values.T
ax.plot(data)
ax.legend(df.index.values, fontsize=16, frameon=False)
# ax.set_xlabel('mtype')
ax.set_ylabel('frequency (Hz)')
# ax.set_title('Frequency')
ax.set_xticks(range(len(folders)))
ax.set_xticklabels(folders, rotation='vertical')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

### Exercise #1

Another test we presented in the lecture is to disable inhibition. Launch a simulation with the same parameters of the simulation "minis" of above but without inhibition. To recapitulate, the new simulation should be:
- all neurons are simulated
- duration of 1000 ms
- projections (Schaffer collaterals) included
- spontaneous synaptic activity (minis) at 0.01 Hz
- set weights=0 for connections from interneurons to pyramidal cells

Compute the distributions of mean frequencies from all the cells in "minis" and "minis without inhibition" (remember to exclude the first 200 ms from the analysis). Store means and standand deviations of the two distributions in a list called ans_1.

In [ ]:
# This is to generate the answers to paste in the submission box below.
# After you defined the variables with your answers, run this cell and the next cell, and copy-paste the output into the box below
print(json.dumps(dict([("ans_1", ans_1)])))

In [ ]:
!pip -q install -i https://bbpteam.epfl.ch/repository/devpi/simple/ single-cell-mooc-client==0.0.5 
import single_cell_mooc_client as sc_mc
s = sc_mc.Submission(hideToken=True)

In [ ]:
s.show_submission(ROLLBACK, TOKEN)